In [20]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import set_config

set_config(display="text")
set_config(print_changed_only=True)


random.seed(10)

In [21]:
x_train = np.load("../../data/x_train.npy")
y_train = np.load("../../data/y_train.npy")


train_important = x_train[:, [5, 9, 100, 101, 102, 103, 104, 105]]
y = pd.DataFrame(y_train)
y = y.values.ravel()

In [22]:
random.seed(423)
random_states = [random.randint(0, 10000) for i in range(80)]
random_states

accuracies_svm_with_interactions = []

for random_state in random_states:
    X_train, X_test, y_train, y_test = train_test_split(
        train_important, y, test_size=0.11, random_state=random_state
    )

    svm_with_interactions = SVC(kernel="poly", degree=2)
    svm_with_interactions.fit(X_train, y_train)
    accuracy = svm_with_interactions.score(X_test, y_test)
    accuracies_svm_with_interactions.append(accuracy)

print(f"Accuracy: {np.mean(accuracies_svm_with_interactions)}")

Accuracy: 0.7260511363636363


0, 2, 3, 4, 5, 6, 8, 9, 100, 101, 102, 103, 104, 105, 403, 466 - 56.7%

0, 2, 3, 4, 5, 6, 8, 9, 100, 101, 102, 103, 104, 105, 403, 466 - 60.5%

0, 2, 3, 4, 5, 6, 8, 9, 100, 101, 102, 103, 104, 105, 403 - 60.5%

0, 2, 3, 4, 5, 6, 8, 9, 100, 101, 102, 103, 104, 105 - 72.5% 

0, 5, 6, 8, 9, 100, 101, 102, 103, 104, 105 - 72.5%

0, 5, 6, 9, 100, 101, 102, 103, 104, 105 - 72.5%

0, 5, 9, 100, 101, 102, 103, 104, 105 - 72.4%

0, 5, 9, 100, 101, 102, 103, 104, 105, 466 - 58.8%

0, 5, 9, 101, 102, 103, 104, 105 - 71.1%

0, 5, 9, 100, 102, 103, 104, 105 - 70.1%

0, 5, 9, 100, 101, 103, 104, 105 - 68.8%

0, 5, 9, 100, 101, 102, 104, 105 - 70.5%

0, 5, 9, 100, 101, 102, 103, 105 - 70.1%

0, 5, 9, 100, 101, 102, 103, 104 - 68.9%

0, 5, 9, 100, 101, 102, 103, 104, 105, 75 - 72.1%

0, 5, 9, 100, 101, 102, 103, 104, 105, 220 - 71.9%

0, 5, 9, 100, 101, 102, 103, 104, 105, 285 - 71.9%

0, 5, 9, 100, 101, 102, 103, 104, 105, 323 - 72.1%

**5, 9, 100, 101, 102, 103, 104, 105 - 72.6%**

9, 100, 101, 102, 103, 104, 105 - 70.8%

5, 100, 101, 102, 103, 104, 105 - 70.6%

5, 9, 101, 102, 103, 104, 105 - 69.9%

5, 9, 100, 102, 103, 104, 105 - 69.9%

5, 9, 100, 101, 103, 104, 105 - 68.8%

5, 9, 100, 101, 102, 104, 105 - 70.6%

5, 9, 100, 101, 102, 103, 105 - 69.8%

5, 9, 100, 101, 102, 103, 104 - 69.1%

## What level of final performance expect from 72% acc model

In [178]:
x_train = np.load("../../data/x_train.npy")
y_train = np.load("../../data/y_train.npy")


x_val = np.load("../../data/x_val.npy")
y_val = np.load("../../data/y_val.npy")

train_important = x_train[:, [102, 103, 105]]
val_important = x_val[:, [102, 103, 105]]
X_train = train_important

In [183]:
cv_folds = 5
np.random.seed(3127)
idx = np.array(range(X_train.shape[0]))
np.random.shuffle(idx)
svm_results = []


idx = idx % cv_folds
for j in range(cv_folds):
    X_train_cv = X_train[idx != j]
    y_train_cv = y_train[idx != j]
    X_val_cv = X_train[idx == j]
    y_val_cv = y_train[idx == j]

    svm = SVC(probability=True, random_state=44)
    svm.fit(X_train_cv, y_train_cv)

    y_pred = svm.predict_proba(X_val_cv)[:, 1]

    n = len(y_pred)
    top_02 = np.argsort(y_pred)[::-1][: n // 5]
    y_pred = np.zeros(n)
    y_pred[top_02] = 1

    svm_results.append(sum(y_pred.astype(int) & y_val_cv.astype(int)) / sum(y_pred))

print(
    np.mean(svm_results) * 10000 - len(train_important[0]) * 200,
    np.std(svm_results) * 10000,
)

7025.0 205.39595906443736


In [184]:
X_train

array([[ 0.22400722, -1.47985066, -0.50026472],
       [ 2.12799427,  0.23407729,  0.16675664],
       [-0.17922693, -0.08115805,  1.4506807 ],
       ...,
       [-0.58964283,  0.81455566, -0.98207745],
       [ 0.04939944, -0.31992458, -1.61985734],
       [-1.13389733, -1.07901437, -2.05424687]])